In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
#tensors are 4D array
#Convert MNIST dataset into tensor(4D-no.of images,height,width,color)
transform=transforms.ToTensor()


In [3]:
train_data=datasets.MNIST(root="/cnn_data",train=True,download=True,transform=transform)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4845866.35it/s] 


Extracting /cnn_data\MNIST\raw\train-images-idx3-ubyte.gz to /cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 135856.11it/s]


Extracting /cnn_data\MNIST\raw\train-labels-idx1-ubyte.gz to /cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1262156.41it/s]


Extracting /cnn_data\MNIST\raw\t10k-images-idx3-ubyte.gz to /cnn_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2214405.30it/s]

Extracting /cnn_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to /cnn_data\MNIST\raw



In [4]:
test_data=datasets.MNIST(root="/cnn_data",train=False,download=True,transform=transform)


In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10,shuffle=False)

In [13]:
#Define The CNN
conv1=nn.Conv2d(1,6,3,1)
conv2=nn.Conv2d(6,16,3,1)


In [9]:
x_train,y_train=train_data[0]


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

Seeing How only one image is passed through layers

In [11]:
X=x_train.view(1,1,28,28)

In [14]:
X=F.relu(conv1(X))


In [15]:
X.shape

torch.Size([1, 6, 26, 26])

In [16]:
X=F.max_pool2d(X,2,2)

In [17]:
X.shape

torch.Size([1, 6, 13, 13])

In [18]:
X=F.relu(conv2(X))

In [19]:
X.shape

torch.Size([1, 16, 11, 11])

In [20]:
X=F.max_pool2d(X,2,2)
X.shape

torch.Size([1, 16, 5, 5])

Making the Model itself

In [26]:
class ConventionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,3,1)
        self.conv2=nn.Conv2d(6,16,3,1)
        #Fully connected layers
        self.fc1=nn.Linear(5*5*16,120)#5*5*16= 16 outputs of each 5 height and width
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,X):
        X=F.relu(self.conv1(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv2(X))
        X=F.max_pool2d(X,2,2)
        X=X.view(-1,5*5*16)
        #Fully connected
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=self.fc3(X)
        return F.log_softmax(X,dim=1)

In [27]:
#Create the instance of the class
torch.manual_seed(41)

model= ConventionalNetwork()
model

ConventionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)